* Author: HZQ
* Last modified: 2018/12/4

# Main Parts

## Import required packages

In [0]:
def name():
    pass

## prepared process


### read data online

### read data offline

## model training

### baseline model: (e.g Linear regression)

## model validation


# Undertesting Parts